### Data Preparation

##### Here we prepare the data for model training and inference. We load the [Rotten Tomatoes dataset](https://huggingface.co/datasets/rotten_tomatoes), from Hugging Face's *datasets* library, and then create features from it using the [*Tokenizer*](https://huggingface.co/docs/transformers/main_classes/tokenizer) class from the [*transformers*](https://huggingface.co/docs/transformers/index) library. The dataset with features is then saved as parquet files in the Databricks File System.

Import the necessary packages.

In [0]:
import os
import pandas as pd

from pyspark.sql.functions import pandas_udf, monotonically_increasing_id
from pyspark.sql.types import IntegerType, ArrayType, StructType, StructField

from transformers import AutoConfig, AutoTokenizer
from datasets import load_dataset

Define the number of workers in the cluster and the folders to save the prepared data under the Databricks File System.

In [0]:
num_workers = 8

train_data_folder = '/train_data'
val_data_folder = '/val_data'
test_data_folder = '/test_data'

We load the training, validation, and testing partitions from the *Rotten Tomatoes* dataset and save the corresponding *Dataset* objects as temporary parquet files. This makes it easier to load them as Spark *DataFrame* objects later.

In [0]:
ds_train = load_dataset('rotten_tomatoes', split='train')
ds_val = load_dataset('rotten_tomatoes', split='validation')
ds_test = load_dataset('rotten_tomatoes', split='test')

ds_train.to_parquet('/dbfs/tmp/train_data.pq')
ds_val.to_parquet('/dbfs/tmp/val_data.pq')
ds_test.to_parquet('/dbfs/tmp/test_data.pq')

Using custom data configuration default
Reusing dataset rotten_tomatoes (/root/.cache/huggingface/datasets/rotten_tomatoes/default/1.0.0/40d411e45a6ce3484deed7cc15b82a53dad9a72aafd9f86f8f227134bec5ca46)
Using custom data configuration default
Reusing dataset rotten_tomatoes (/root/.cache/huggingface/datasets/rotten_tomatoes/default/1.0.0/40d411e45a6ce3484deed7cc15b82a53dad9a72aafd9f86f8f227134bec5ca46)
Using custom data configuration default
Reusing dataset rotten_tomatoes (/root/.cache/huggingface/datasets/rotten_tomatoes/default/1.0.0/40d411e45a6ce3484deed7cc15b82a53dad9a72aafd9f86f8f227134bec5ca46)
Out[3]: 135968

Here we load the training, validation, and testing partitions as Spark *DataFrame* objects and repartition the data by the number of workers in our cluster. We do this to make sure all workers in the cluster are used to process the data and to illustrate the parallel processing with *Pandas UDF* below, given that the data is relatively small. We then include a new column, *id*, as a unique identifier in each. This will later help to join the model predictions with the original datasets, as needed.

In [0]:
os.makedirs(train_data_folder, exist_ok = True)
os.makedirs(val_data_folder, exist_ok = True)
os.makedirs(test_data_folder, exist_ok = True)

sdf_train = spark.read.parquet('/tmp/train_data.pq').repartition(num_workers)
sdf_train = sdf_train.withColumn('id', monotonically_increasing_id())
sdf_train.write.parquet(os.path.join('dbfs', train_data_folder, 'train_data'), mode='overwrite')

sdf_val = spark.read.parquet('/tmp/val_data.pq').repartition(num_workers)
sdf_val = sdf_val.withColumn('id', monotonically_increasing_id())
sdf_val.write.parquet(os.path.join('dbfs', val_data_folder, 'val_data'), mode='overwrite')

sdf_test = spark.read.parquet('/tmp/test_data.pq').repartition(num_workers)
sdf_test = sdf_test.withColumn('id', monotonically_increasing_id())
sdf_test.write.parquet(os.path.join('dbfs', test_data_folder, 'test_data'), mode='overwrite')

display(sdf_train.select(sdf_train.columns).limit(10))

text,label,id
"there's a disreputable air about the whole thing , and that's what makes it irresistible .",1,0
it's like an all-star salute to disney's cheesy commercialism .,0,1
"if the first men in black was money , the second is small change . but it still jingles in the pocket . it's fun lite .",1,2
this movie is about the worst thing chan has done in the united states .,0,3
"it's always disappointing when a documentary fails to live up to -- or offer any new insight into -- its chosen topic . unfortunately , that's precisely what arthur dong's family fundamentals does .",0,4
"even if you have no interest in the gang-infested , east-vs . -west coast rap wars , this modern mob music drama never fails to fascinate .",1,5
"a completely spooky piece of business that gets under your skin and , some plot blips aside , stays there for the duration .",1,6
"a gushy episode of "" m * a * s * h "" only this time from an asian perspective .",0,7
"the aaa of action , xxx is a blast of adrenalin , rated eee for excitement . and vin diesel is the man .",1,8
"the period -- swinging london in the time of the mods and the rockers -- gets the once-over once again in gangster no . 1 , but falls apart long before the end .",0,9


Here is where we define a function to create the features needed by the model. We choose [Microsoft's *DeBERTa v3*](https://github.com/microsoft/DeBERTa) model, which is [available as a pre-trained model](https://huggingface.co/microsoft/deberta-v3-base) from the Hugging Face model catalog. We use Hugging Face's [*Auto Classes*](https://huggingface.co/docs/transformers/model_doc/auto#auto-classes) to instantiate the appropriate *Tokenizer* object from the model chosen.

The *tokenization* process is performed in parallel, over the Spark DataFrame, using Spark's [*Pandas UDF*](https://docs.microsoft.com/en-us/azure/databricks/spark/latest/spark-sql/udf-python-pandas) functionality.

In [0]:
model_type = 'microsoft/deberta-v3-base'
tokenizer = AutoTokenizer.from_pretrained(model_type)
max_length = 128

schema = StructType([
    StructField('input_ids', ArrayType(IntegerType())),
    StructField('token_type_ids', ArrayType(IntegerType())),
    StructField('attention_mask', ArrayType(IntegerType()))
])

@pandas_udf(schema)
def tokenize(text: pd.Series) -> pd.DataFrame:
    res = []
    for t in text:
        tokens = tokenizer(t, max_length=max_length, truncation=True, padding='max_length')
        res.append({'input_ids': tokens['input_ids'], 'token_type_ids': tokens['token_type_ids'], 'attention_mask': tokens['attention_mask']})
    return pd.DataFrame(res)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.9/site-packages/transformers/convert_slow_tokenizer.py:434: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


The *tokenization* function defined above is then executed through the *select* function from the *DataFrame* API. It returns the three types of inputs needed by the model: *input_ids*, *token_type_ids*, and *attention_mask*.

In [0]:
sdf_tokens_train = sdf_train.select('id', 'label', tokenize('text').alias('tokens'))
sdf_tokens_val = sdf_val.select('id', 'label', tokenize('text').alias('tokens'))
sdf_tokens_test = sdf_test.select('id', 'label', tokenize('text').alias('tokens'))

display(sdf_tokens_train.select(sdf_tokens_train.columns).limit(10))

id,label,tokens
0,1,"List(List(1, 343, 280, 268, 266, 111624, 925, 314, 262, 783, 576, 366, 263, 272, 280, 268, 339, 682, 278, 23896, 323, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0), List(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0), List(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0))"
1,0,"List(List(1, 278, 280, 268, 334, 299, 305, 271, 5104, 24779, 264, 36094, 280, 268, 22664, 1613, 2934, 323, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0), List(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0), List(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0))"
2,1,"List(List(1, 337, 262, 362, 842, 267, 937, 284, 580, 366, 262, 567, 269, 536, 575, 323, 304, 278, 449, 47548, 268, 267, 262, 4346, 323, 278, 280, 268, 785, 36938, 323, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0), List(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0), List(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0))"
3,0,"List(List(1, 291, 1421, 269, 314, 262, 3130, 576, 46530, 303, 619, 267, 262, 9074, 1603, 323, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

We split the *tokenization* results into separate columns in the corresponding *DataFrame* objects. These are our *featurized* data.

In [0]:
sdf_tokens_train = sdf_tokens_train.withColumns({'input_ids': sdf_tokens_train['tokens'].getItem('input_ids'),
                                                 'token_type_ids': sdf_tokens_train['tokens'].getItem('token_type_ids'),
                                                 'attention_mask': sdf_tokens_train['tokens'].getItem('attention_mask')}).drop('tokens')

sdf_tokens_val = sdf_tokens_val.withColumns({'input_ids': sdf_tokens_val['tokens'].getItem('input_ids'),
                                             'token_type_ids': sdf_tokens_val['tokens'].getItem('token_type_ids'),
                                             'attention_mask': sdf_tokens_val['tokens'].getItem('attention_mask')}).drop('tokens')

sdf_tokens_test = sdf_tokens_test.withColumns({'input_ids': sdf_tokens_test['tokens'].getItem('input_ids'),
                                               'token_type_ids': sdf_tokens_test['tokens'].getItem('token_type_ids'),
                                               'attention_mask': sdf_tokens_test['tokens'].getItem('attention_mask')}).drop('tokens')

display(sdf_tokens_train.select(sdf_tokens_train.columns).limit(10))

id,label,input_ids,token_type_ids,attention_mask
0,1,"List(1, 343, 280, 268, 266, 111624, 925, 314, 262, 783, 576, 366, 263, 272, 280, 268, 339, 682, 278, 23896, 323, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)","List(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)","List(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)"
1,0,"List(1, 278, 280, 268, 334, 299, 305, 271, 5104, 24779, 264, 36094, 280, 268, 22664, 1613, 2934, 323, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)","List(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)","List(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)"
2,1,"List(1, 337, 262, 362, 842, 267, 937, 284, 580, 366, 262, 567, 269, 536, 575, 323, 304, 278, 449, 47548, 268, 267, 262, 4346, 323, 278, 280, 268, 785, 36938, 323, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)","List(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)","List(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)"
3,0,"List(1, 291, 1421, 269, 314, 262, 3130, 576, 46530, 303, 619, 267, 262, 9074, 1603, 323, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

We then write the *featurized* data as parquet files in the Databricks File System.

In [0]:
sdf_tokens_train.write.parquet(os.path.join('dbfs', train_data_folder, 'tokens_train_data'), mode='overwrite')
sdf_tokens_val.write.parquet(os.path.join('dbfs', val_data_folder, 'tokens_val_data'), mode='overwrite')
sdf_tokens_test.write.parquet(os.path.join('dbfs', test_data_folder, 'tokens_test_data'), mode='overwrite')